In [1]:
# coding: utf-8

# In[1]:

import numpy as np
import cmath
from numpy.lib import scimath
import os, sys
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl
import subprocess
scriptpath = "./colormap-master/"
sys.path.append(os.path.abspath(scriptpath))
import colormaps as cmaps
plt.register_cmap(name='viridis', cmap=cmaps.viridis)
plt.set_cmap(cmaps.viridis)
#plt.register_cmap(name='plasma', cmap=cmaps.plasma)
#plt.set_cmap(cmaps.plasma)

matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [2]:
#fig2 = plt.figure(figsize=(5,5))
#============================================
def openFile (B, eta):
    if B in ("0"):
        path = '/media/d3/FUM_PROJ/nrk_no_AD/consMesh/2001_50.0/b' + B + '/brf_results/'
    else:
        path = '/media/d3/FUM_PROJ/nrk_AD/consMesh/2000_50.0/b' + B + '/bc4310/ADlimit100/eta' + eta + '/brf_results/'
    brfFiles = subprocess.check_output(['ls'], cwd=path)
    #print type(brfFiles)
    listOfFiles = brfFiles.split()
    FastestGrowingFile = listOfFiles[-1]
    FastestGrowingFileSplited = FastestGrowingFile.split("_")
    omega2 = FastestGrowingFileSplited[0]
    FastestGrowingFile = path + FastestGrowingFile
    aaaa = 'cp ' + FastestGrowingFile + ' ~/mmm'
    subprocess.call(aaaa, shell=True)
    print "Open file:\n",FastestGrowingFile
    r, f, phi, w, dphi, k, bz, dbz = np.loadtxt(FastestGrowingFile, unpack=True, usecols = (  0, 1, 2, 3, 4, 5, 6, 7 ))
    return r, f, phi, w, dphi, k, bz, dbz, omega2
#===========================================
def density (r):
    rho = 1./((1. + r*r/8.)**2.)
    return rho

In [59]:
#===========================================
def makePlot (B, eta, plotNumber):
       
    r, f, phi, w, dphi, k, bz, dbz, omega2 = openFile (B, eta) # Open file and get data
    
    #==============
    #ax2 = fig2.add_subplot(1,1,1)
    #ax2.set_ylim(-0.05, 1.05)
    #ax2.set_xlim(-0.05, 10)
    #ax2.set_xlabel("r")
    #ax2.set_ylabel("f")
    #P2 = ax2.plot(r,f, label=B)
    #ax2.legend()
    #==============
        
    print "omega2 = ", omega2
    #upperBound = int(2.*cmath.pi/k[0]*2000/50)
    upperBound = 800
    print "upperBound = ", upperBound
    r = r[0:upperBound:stepR]; f = f[0:upperBound:stepR]; phi = phi[0:upperBound:stepR]
    w = w[0:upperBound:stepR]; dphi = dphi[0:upperBound:stepR]; k = k[0:upperBound:stepR]
    bz = bz[0:upperBound:stepR]; dbz = dbz[0:upperBound:stepR]
    SizeOfr = r.size
    print 'Size Of r = ',SizeOfr # print r[SizeOfr-1]; print r
    omega2 = float (omega2)
    omega = scimath.sqrt(-omega2)
    print "omega = ", omega
    z = np.linspace(-r[SizeOfr-1], r[SizeOfr-1], 2*SizeOfr)
    #z = np.linspace(-r[(SizeOfr-1)/2], r[(SizeOfr-1)/2], SizeOfr)
    print 'Size Of z = ', z.size
    
    rho = np.zeros(SizeOfr)
    vz = np.zeros(SizeOfr)
    vr = np.zeros(SizeOfr)
   
    rho = density (r)
    vr = w / (I * omega)
    vz = k/omega*(pd/rho*f+phi)

    u1z = np.zeros((SizeOfr,z.size))
    u1r = np.zeros((SizeOfr,z.size))
    rho1 = np.zeros((SizeOfr,z.size))
    rhoT = np.zeros((SizeOfr,z.size))
    for i in range (SizeOfr):
        for n in range (z.size):
        #u1r[i][n] = ( vr[i] * ( cmath.cos(k[i]*z[n]) + I*cmath.sin(k[i]*z[n])) ).real
        #u1z[i][n] = ( vz[i] * ( cmath.cos(k[i]*z[n]) + I*cmath.sin(k[i]*z[n])) ).real
            u1r[i][n] = ( vr[i] * ( cmath.cos(k[i]*z[n]) ) ).real
            u1z[i][n] = ( vz[i] * ( I*cmath.sin(k[i]*z[n]) ) ).real
        #print i, n, r[i], z[n], vr[i], vz[i], u1r[i][n], u1z[i][n]
            rho1[i][n] = ( f[i] * ( cmath.cos(k[i]*z[n]) + I*cmath.sin(k[i]*z[n])) ).real     
            rhoT[i][n] = rho[i] + rho1[i][n]

    (Z,R) = np.meshgrid(z,r)
    print z.shape, Z.shape

    #ax = fig.add_subplot(1,4,plotNumber)
    ax = fig1.add_axes([0.05+(plotNumber-1)*0.21, 0.05, 0.20, 0.9])
    ax.set_ylim(-(cmath.pi/k[0]), cmath.pi/k[0])
    #plt.ylim(-11, 11)
    ax.set_xlim(-0.01, 6)
    CS = ax.contourf(R,Z,rhoT,300)
    ax.set_yticklabels('')
    ax.yaxis.set_ticks_position('none')
    ax.set_xticklabels('')
    ax.xaxis.set_ticks_position('none')
    etaFloat = float(eta)
    etaStr = str(etaFloat)
    bLabel = "B =" + B
    etaLabel = "eta = " + etaStr + "\n" + bLabel
    plt.text(0.3, 0.08, etaLabel, horizontalalignment='center',verticalalignment='center',
             transform = ax.transAxes, color = 'k', backgroundcolor='floralwhite')
    #ax.set_xlabel("r")
    #, rotation='vertical')
    #plt.clabel(CS, inline=1, fontsize=10)
    if (plotNumber == 4):
         colorbar_ax = fig1.add_axes([0.89, 0.05, 0.015, 0.9])
         CB = fig1.colorbar(CS, cax=colorbar_ax)#, shrink=0.5, extend='both')          # vertical colour bar
         CB.set_ticks([0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2])
         CB.set_ticklabels([0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2])
         CB.set_label(r'$\rho\quad[\rho_c]$' )
    ax.quiver(R,Z,u1r,u1z,scale=12,color='w')
    
    for c in CS.collections: ## to hide bad white lines in counterfill
         c.set_edgecolor("face")
        
    #if (plotNumber != 1):
         #ax.set_yticklabels('')
    #plt.show()
    #plt.close()

In [62]:
#====================
I = 0 + 1j
stepR = 30
pd = 1.

fig1 = plt.figure(figsize=(10,5), dpi=300)
fig1.text(0.5, 0.02, 'r', ha='center')
fig1.text(0.02, 0.5, 'z', va='center')
#====================
B = "0"; eta = "0"; plotNumber = 1
makePlot (B, eta, plotNumber)
B = "1.0"; eta = "0"; plotNumber = 2
makePlot (B, eta, plotNumber)
B = "2.0"; eta = "0"; plotNumber = 3
makePlot (B, eta, plotNumber)
B = "5.0"; eta = "0"; plotNumber = 4
makePlot (B, eta, plotNumber)
#====================
#plt.show()
plt.savefig('u_field.eps')
plt.close(fig1)
#====================
fig1 = plt.figure(figsize=(10,5), dpi=300)
fig1.text(0.5, 0.02, 'r', ha='center')
fig1.text(0.02, 0.5, 'z', va='center')
#====================
B = "1.0"; eta = "00010.00000"; plotNumber = 1
makePlot (B, eta, plotNumber)
B = "1.0"; eta = "00100.00000"; plotNumber = 2
makePlot (B, eta, plotNumber)
B = "2.0"; eta = "00100.00000"; plotNumber = 3
makePlot (B, eta, plotNumber)
B = "5.0"; eta = "00100.00000"; plotNumber = 4
makePlot (B, eta, plotNumber)
#====================
#plt.show()
plt.savefig('u_field_big_eta.eps')
plt.close(fig1)

cpEps = 'cp ./*.eps /media/d1/Mohammad/PROJECTS/FUM/docs/reports/images'
subprocess.call(cpEps, shell=True)


Open file:
/media/d3/FUM_PROJ/nrk_no_AD/consMesh/2001_50.0/b0/brf_results/0.116800_0.286847
omega2 =  0.116800
upperBound =  800
Size Of r =  27
omega =  0.341760149813j
Size Of z =  54
(54,) (27, 54)
Open file:
/media/d3/FUM_PROJ/nrk_AD/consMesh/2000_50.0/b1.0/bc4310/ADlimit100/eta0/brf_results/0.092000_0.311109
omega2 =  0.092000
upperBound =  800
Size Of r =  27
omega =  0.303315017762j
Size Of z =  54
(54,) (27, 54)
Open file:
/media/d3/FUM_PROJ/nrk_AD/consMesh/2000_50.0/b2.0/bc4310/ADlimit100/eta0/brf_results/0.089800_0.312772
omega2 =  0.089800
upperBound =  800
Size Of r =  27
omega =  0.299666481275j
Size Of z =  54
(54,) (27, 54)
Open file:
/media/d3/FUM_PROJ/nrk_AD/consMesh/2000_50.0/b5.0/bc4310/ADlimit100/eta0/brf_results/0.089000_0.316707
omega2 =  0.089000
upperBound =  800
Size Of r =  27
omega =  0.298328677804j
Size Of z =  54
(54,) (27, 54)
Open file:
/media/d3/FUM_PROJ/nrk_AD/consMesh/2000_50.0/b1.0/bc4310/ADlimit100/eta00010.00000/brf_results/0.091000_0.390300
omega2

0